# Marginal Working Group
This is a program that will analyze the margin for JPL IPE MWG.


Import file from the directory you are working in. 
I will be using pandas to organize the .csv file that we are dealing with. df = Data Frame

In [10]:
import pandas as pd
import urllib
import os, sys, json, re
from math import *
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np


filename = "sol2170.csv"
f = open(filename, "r")
df_kb = pd.read_csv(f)


# with open(filename) as fp:
#     data = json.load(fp)

In [11]:
# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [12]:
df_kb.dropna(axis = 0, how = "any", inplace = True)
df_kb.reset_index(drop = True, inplace = True)
df_kb

,recordType,sessionId,sessionHost,name,module,level,eventId,fromSse,realtime,sclk,scet,lst,ert,rct,message,metadataKeywordList,metadataValuesList,sessionDssId,stringId
0,Evr,24945,mslsmsampcs1,FSM_EVR_FLIGHT_SOFTWARE_VERSION,fsm,ACTIVITY_HI,790726143,False,False,5.900926e+08,2018-256T07:09:38.973,SOL-2170M01:32:29.918,2018-256T07:47:42.613,2018-256T07:47:40.257,"FSW version 0x0c7365fb, fsw_release_string=R12...","[(CategorySequenceId),(SequenceId),(TaskName)]","[(0),(RT:1),(fsm???)]",0,B
1,Evr,24945,mslsmsampcs1,MEM_EVR_BPMM_CRC32,mem,ACTIVITY_LO,1280031383,False,False,5.900926e+08,2018-256T07:09:38.973,SOL-2170M01:32:29.918,2018-256T07:47:43.639,2018-256T07:47:40.551,"BPMM addr [0xbf898030], BPMM CRC-32 [0xc71c0011].","[(CategorySequenceId),(SequenceId),(TaskName)]","[(0),(RT:0),(fsm???)]",0,B
2,Evr,24945,mslsmsampcs1,FSM_EVR_FSW_BUILD_TIME,fsm,ACTIVITY_LO,795939399,False,False,5.900926e+08,2018-256T07:09:38.973,SOL-2170M01:32:29.918,2018-256T07:47:43.639,2018-256T07:47:40.551,"FSW build time: month 8 day 14 year 2014, hour...","[(CategorySequenceId),(SequenceId),(TaskName)]","[(1),(RT:2),(fsm???)]",0,B
3,Evr,24945,mslsmsampcs1,FSM_EVR_FSW_ELAPSED_TIME,fsm,ACTIVITY_LO,798985376,False,False,5.900926e+08,2018-256T07:09:38.973,SOL-2170M01:32:29.918,2018-256T07:47:43.639,2018-256T07:47:40.552,"Elapsed time since boot is 20 second(s), with ...","[(CategorySequenceId),(SequenceId),(TaskName)]","[(2),(RT:3),(fsm???)]",0,B
4,Evr,24967,mslsmsampcs1,AUT_EVR_BUFFERS_AVAIL,aut,DIAGNOSTIC,181449080,False,False,5.900926e+08,2018-256T07:09:38.973,SOL-2170M01:32:29.918,2018-258T10:27:02.672,2018-258T10:19:37.313,"AUT got memory buffers: mem_rcvd 2097152, reco...","[(CategorySequenceId),(SequenceId),(TaskName)]","[(0),(RT:4),(fsm???)]",0,B
5,Evr,24967,mslsmsampcs1,AUT_EVR_AUT_MEMORY_CHECK,aut,DIAGNOSTIC,180144718,False,False,5.900926e+08,2018-256T07:09:38.973,SOL-2170M01:32:29.918,2018-258T10:27:02.672,2018-258T10:19:37.313,"AUT memory has no data. recoverable FALSE, san...","[(CategorySequenceId),(SequenceId),(TaskName)]","[(1),(RT:5),(fsm???)]",0,B
6,Evr,24967,mslsmsampcs1,AUT_EVR_BUFFERS_AS_REQUESTED,aut,DIAGNOSTIC,172867028,False,False,5.900926e+08,2018-256T07:09:39.005,SOL-2170M01:32:29.949,2018-258T10:27:02.672,2018-258T10:19:37.313,AUT got all requested buffers.,"[(CategorySequenceId),(SequenceId),(TaskName)]","[(2),(RT:6),(fsm???)]",0,B
7,Evr,24967,mslsmsampcs1,AUT_EVR_INIT_TYPE3,aut,DIAGNOSTIC,169478013,False,False,5.900926e+08,2018-256T07:09:39.005,SOL-2170M01:32:29.949,2018-258T10:27:02.672,2018-258T10:19:37.313,AUT init type AUT_INIT_CLEAN.,"[(CategorySequenceId),(SequenceId),(TaskName)]","[(3),(RT:7),(fsm???)]",0,B
8,Evr,24967,mslsmsampcs1,AUT_EVR_INIT_DONE,aut,DIAGNOSTIC,177031458,False,False,5.900926e+08,2018-256T07:09:39.005,SOL-2170M01:32:29.949,2018-258T10:27:02.672,2018-258T10:19:37.313,AUT done with aut_init: ready for aut_finish_i...,"[(CategorySequenceId),(SequenceId),(TaskName)]","[(4),(RT:8),(fsm???)]",0,B
9,Evr,24967,mslsmsampcs1,CRCMGR_EVR_CRCC_PRESENT,crcmgr,DIAGNOSTIC,459361291,False,False,5.900926e+08,2018-256T07:09:39.005,SOL-2170M01:32:29.949,2018-258T10:27:02.672,2018-258T10:19:37.313,"CRCMGR CRCC Present, board type FLT, serial nu...","[(CategorySequenceId),(SequenceId),(TaskName)]","[(5),(RT:9),(fsm???)]",0,B


In [13]:
#Making sure right file type

In [14]:
df_kb_dropped = df_kb.drop(columns=['recordType', 'sessionHost', 'sessionId','fromSse', 'eventId', 'realtime', 'sclk', 'scet',
                    'ert', 'rct', 'metadataKeywordList','metadataValuesList',
                    'sessionDssId','stringId'])

df_kb_dropped

,name,module,level,lst,message
0,FSM_EVR_FLIGHT_SOFTWARE_VERSION,fsm,ACTIVITY_HI,SOL-2170M01:32:29.918,"FSW version 0x0c7365fb, fsw_release_string=R12..."
1,MEM_EVR_BPMM_CRC32,mem,ACTIVITY_LO,SOL-2170M01:32:29.918,"BPMM addr [0xbf898030], BPMM CRC-32 [0xc71c0011]."
2,FSM_EVR_FSW_BUILD_TIME,fsm,ACTIVITY_LO,SOL-2170M01:32:29.918,"FSW build time: month 8 day 14 year 2014, hour..."
3,FSM_EVR_FSW_ELAPSED_TIME,fsm,ACTIVITY_LO,SOL-2170M01:32:29.918,"Elapsed time since boot is 20 second(s), with ..."
4,AUT_EVR_BUFFERS_AVAIL,aut,DIAGNOSTIC,SOL-2170M01:32:29.918,"AUT got memory buffers: mem_rcvd 2097152, reco..."
5,AUT_EVR_AUT_MEMORY_CHECK,aut,DIAGNOSTIC,SOL-2170M01:32:29.918,"AUT memory has no data. recoverable FALSE, san..."
6,AUT_EVR_BUFFERS_AS_REQUESTED,aut,DIAGNOSTIC,SOL-2170M01:32:29.949,AUT got all requested buffers.
7,AUT_EVR_INIT_TYPE3,aut,DIAGNOSTIC,SOL-2170M01:32:29.949,AUT init type AUT_INIT_CLEAN.
8,AUT_EVR_INIT_DONE,aut,DIAGNOSTIC,SOL-2170M01:32:29.949,AUT done with aut_init: ready for aut_finish_i...
9,CRCMGR_EVR_CRCC_PRESENT,crcmgr,DIAGNOSTIC,SOL-2170M01:32:29.949,"CRCMGR CRCC Present, board type FLT, serial nu..."


In [47]:
#Write out
# df_kb.to_csv(r'output.csv')

In [7]:
length = len(df_kb)
result = []
for i in range(0, length):
    if df_kb.loc[i, 'level'] == "ACTIVITY_LO" or df_kb.loc[i, 'level'] == "ACTIVITY_HI":
        df2 = pd.DataFrame(df_kb.iloc[i])
        result.append(df2)

result

[                                                                     0
 recordType                                                         Evr
 sessionId                                                        24945
 sessionHost                                               mslsmsampcs1
 name                                   FSM_EVR_FLIGHT_SOFTWARE_VERSION
 module                                                             fsm
 level                                                      ACTIVITY_HI
 eventId                                                      790726143
 fromSse                                                          False
 realtime                                                         False
 sclk                                                       5.90093e+08
 scet                                             2018-256T07:09:38.973
 lst                                              SOL-2170M01:32:29.918
 ert                                              2018-256T07:47

In [78]:
for uniqueId in data.keys():
    amountMarginUsed = round((data[uniqueId]['planMarginDur'] - data[uniqueId]['actMarginDur'])/60,3)
    print(uniqueId, amountMarginUsed)

sol2170_sub_00170 28.381
sol2170_sub_01170 -1.446
sol2171_sub_00171 -7.907
sol2209_sub_00209 -2.901
sol2209_sub_01209 -1.894
sol2209_sub_02209 -1.66
sol2209_sub_03209 -1.893
sol2209_sub_04209 -2.012
sol2209_sub_04210 -1.166
sol2209_sub_05209 -2.01
sol2209_sub_05210 -1.636
sol2209_sub_06209 -1.33
sol2209_sub_06210 -1.649
sol2209_sub_07209 -1.426
sol2209_sub_07210 -1.653
sol2209_sub_08209 -1.772
sol2209_sub_09209 -1.198
sol2209_sub_10209 -1.176
sol2209_sub_11209 -1.118
sol2209_sub_12209 -0.931
sol2209_sub_13209 -0.781
sol2209_sub_14209 -0.792
sol2209_sub_15209 -0.785
sol2209_sub_16209 -1.304
sol2210_sub_00210 -1.274
sol2210_sub_01210 -3.831
sol2210_sub_02210 0.006
sol2211_sub_00211 -0.53
sol2211_sub_01211 -1.012
sol2211_sub_02211 -1.5
sol2212_sub_00212 -0.706
sol2212_sub_01212 -1.47
sol2213_sub_00213 -1.398
sol2213_sub_01213 0.083
sol2213_sub_02213 -7.554
sol2213_sub_03213 -3.187
sol2214_sub_00214 -8.192
sol2215_sub_00215 -2.97
sol2216_sub_00216 -2.288
sol2216_sub_01216 -9.683
sol2217_su